In [2]:
import torch
import numpy as np

def subsequent_mask(size):
    "Mask out subsequent positions." # only the past should influence attention
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8') # np.triu(m, k=0): matrix m, k: diagonal above which to 0: 1=above diagonal
    return torch.from_numpy(subsequent_mask) == 0

if __name__=='__main__':
    print(subsequent_mask(10)[0])

tensor([[ True, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])


In [4]:
# fluent py: sentence
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text): # iter - why sequences are iterable
        self.text = text
        self.words = RE_WORD.findall(text) # findall returns a list of non-overlapping matches of the regex
        
    def __getitem__(self, index):
        return self.words[index] # holds the result of findall
    
    def __len__(self): # needed for the sequence protocol
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # reprlib: utility to generate abbreviated string representations
    
if __name__=='__main__':
    s = Sentence('"The time has come," the Walrus said,')
    print(s)
    for word in s:
        print(word)

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said


In [17]:
# polynomials and RSA
import itertools

class Polynomial:
    
    def __init__(self, coeffs):
        '''init polynomial with a list of coefficients, starting with the lowest power'''
        self.coeffs = list(coeffs)
        
    def __len__(self):
        '''return the degree of the polynomial'''
        return len(self.coeffs)
    
    def __getitem__(self, index):
        '''return the coefficient for the index given'''
        return self.coeffs[index]
        
    def __add__(self, other):
        '''add two polynomials'''
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Polynomial(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
        
    def __radd__(self, other):
        return self + other # delegates to __add__
        
    def val(self, x):
        '''evaluate polynomial at x'''
        res = 0
        cx = 1
        for i in range(len(self)):
            res += self[i] * cx
            cx *= x
        return res
    
    def __str__(self):
        return str(tuple(self))  # build tuple from iterable

if __name__=='__main__':
    p0 = Polynomial([1., 2., 3.])
    print(p0.val(2.))
    p1 = Polynomial([1., 2., 3., 4.])
    p0 += p1
    print(p0.val(2.))
    print(p1.val(2.))
    print(p0)

17.0
66.0
49.0
(2.0, 4.0, 6.0, 4.0)


In [1]:
# fishs in the pond
import random

def fishtag(N, s0, s1, t1):
    tg0 = random.sample(range(N), s0)
    i = 0
    notg = []
    while len(notg) < (s1 - t1):
        ridx = random.randint(0, N)
        i += 1
        if not ridx in tg0:
            notg.append(ridx)
    return i

if __name__=='__main__':
    tst = 0
    for i in range(100):
        tst += fishtag(500, 100, 100, 20)
    print(tst/100) # should be 100

100.25


In [3]:
# sentence iterator
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):  # <1> no getitem: makes it an iterable
        return SentenceIterator(self.words)  # <2> protocol fulfilled: instantiates and returns iterator


class SentenceIterator:

    def __init__(self, words):
        self.words = words  # <3>
        self.index = 0  # <4>

    def __next__(self):
        try:
            word = self.words[self.index]  # <5>
        except IndexError:
            raise StopIteration()  # <6>
        self.index += 1  # <7>
        return word  # <8>

    def __iter__(self):  # <9>
        return self

def main():
    s = Sentence('"The time has come," the Walrus said,')
    print(s)
    for word in s:
        print(word)
    '''import sys
    import warnings
    try:
        filename = sys.argv[1]
        word_number = int(sys.argv[2])
    except (IndexError, ValueError):
        print('Usage: %s <file-name> <word-number>' % sys.argv[0])
        #sys.exit(1)
    with open(filename, 'rt', encoding='utf-8') as text_file:
        s = Sentence(text_file.read())
    for n, word in enumerate(s, 1):
        if n == word_number:
            print(word)
            break
    else:
        warnings.warn('last word is #%d, "%s"' % (n, word))'''

if __name__ == '__main__':
    main()

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said


In [34]:
"""
Sentence: iterate over words using a generator function
"""

import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:  # <1> iterate over words
            yield word  # <2> yield the current word - turns function into a generator
        return  # <3> not needed - function can just fall through
    
def main():
    s = Sentence('"The time has come," the Walrus said,')
    print(s)
    for word in s:
        print(word)

if __name__ == '__main__':
    main()

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said


In [3]:
def gen_123():
    yield 1
    yield 2
    yield 3
    
print(gen_123) # a function object
print(gen_123()) # invoked gen_123() returns a generator object
for i in gen_123(): # generators are iterators that produce the values of the expressions passed to yield
    print(i)
g = gen_123()
print(next(g)) # each next advances to the next yield
print(next(g))
print(next(g))
print(next(g)) # function body complete: StopIteration

<function gen_123 at 0x000002B39DB27A60>
<generator object gen_123 at 0x000002B39DB21468>
1
2
3
1
2
3


StopIteration: 

In [5]:
# FluentPy: more generators

def gen_AB(): # yield makes the function a generator
    print('start')
    yield 'A' # first implicit next() from the for loop prints start and stops at the first yield producing 'A'
    print('continue')
    yield 'B' # next implicit next() stops here
    print('end.')
    
for c in gen_AB():
    print('-->', c)
    #break # just to see where the next() stops

start
--> A


In [17]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # <1> finditer builds an iterator over the matches of RE_WORD
            yield match.group()  # <2> extracts the actually matched text from the MatchObject instance
    
def main():
    s = Sentence('"The time has come," the Walrus said,')
    print(s)
    for word in s:
        print(word)

if __name__ == '__main__':
    main()

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said


In [14]:
# generator expressions

def gen_AB(): # yield makes the function a generator
    print('start')
    yield 'A' # first implicit next() from the for loop prints start and stops at the first yield producing 'A'
    print('continue')
    yield 'B' # next implicit next() stops here
    print('end.')
    
res1 = [x*3 for x in gen_AB()] # list comprehension (eagerly iterates over the items yielded by the generator)
print('looping over res1')
for l in res1:
    print('-->', l)
    #break # just to see where the next() stops
    
res2 = (x*3 for x in gen_AB()) # a generator expression - not yet consumed
print('looping over res2')
for l in res2:
    print('-->', l)
    #break # just to see where the next() stops

start
continue
end.
looping over res1
--> AAA
--> BBB
looping over res2
start
--> AAA
continue
--> BBB
end.


In [18]:
# generator expression
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text)) # generator expression
    
def main():
    s = Sentence('"The time has come," the Walrus said,')
    print(s)
    for word in s:
        print(word)

if __name__ == '__main__':
    main()

Sentence('"The time ha... Walrus said,')
The
time
has
come
the
Walrus
said


In [10]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

class Conv1D(nn.Module):
    def __init__(self, nf, nx):
        super(Conv1D, self).__init__()
        self.nf = nf
        w = torch.empty(nx, nf) # nx by nf
        nn.init.normal_(w, std=0.02) # random numbers, normally distributed
        self.weight = Parameter(w) # normally distributed random weights as parameters
        self.bias = Parameter(torch.zeros(nf)) # no bias

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,) # construct output size: rows of x by nf (size return value is a subclass of tuple)
        print(x.size()[:-1], ' : ', (self.nf,), ' : ', size_out)
        # torch.addmm(beta=1, input, alpha=1, mat1, mat2, out=None)
        # Performs a matrix multiplication of the matrices mat1 and mat2. The matrix input is added to the final result.
        print(x.size(-1)) # -1: number of columns, -2: number of rows
        # multiply x reshaped to (inferred rows, cols of x) with weights (nx=cols of x, nf)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight) # weight is nx by nf, x.size(-1) are cols of x, rows of x.view are inferred (-1)
        #print(x) # x is now (inferred rows of x, nf)
        x = x.view(*size_out) # return a reshaped x : rows of x by nf
        return x

def main():
    c0 = Conv1D(4, 3) # any nf really possible, nx has to match x, however
    print(c0)
    print(c0.forward(torch.Tensor([[1,2,3],[4,5,6]])))
        
if __name__ == '__main__':
    main()

Conv1D()
torch.Size([2])
torch.Size([2])  :  (4,)  :  torch.Size([2, 4])
3
tensor([[0.0114, 0.0153, 0.0089, 0.0966],
        [0.0124, 0.0471, 0.0625, 0.1748]], grad_fn=<ViewBackward>)


In [30]:
import torch
import torch.nn as nn

a = torch.tensor([[2., 3., 1.],[1., 2., 3.]])
print(a.size()) # torch.Size([2])
print(a.size(-1)) # 3
print(a.size(-2)) # 3

m = nn.Softmax(dim=1) # along rows (summing all cols = dim 1)
#input = torch.randn(2, 3)
output = m(a)
print(output)
m = nn.Softmax(dim=-1) # last dimension = sum along rows
output = m(a)
print(output)
m = nn.Softmax(dim=0) # along cols (summing all rows = dim 0)
output = m(a)
print(output)

torch.Size([2, 3])
3
2
tensor([[0.2447, 0.6652, 0.0900],
        [0.0900, 0.2447, 0.6652]])
tensor([[0.2447, 0.6652, 0.0900],
        [0.0900, 0.2447, 0.6652]])
tensor([[0.7311, 0.7311, 0.1192],
        [0.2689, 0.2689, 0.8808]])


In [7]:
# FluentPy: arithmetic progression
import itertools

class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None = infinite series
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin) # value self.begin but of type consistent with .begin and .step
        forever = self.end is None # readability: True if forever
        index = 0
        while forever or result < self.end:
            yield result # current result is produced
            index += 1
            result = self.begin + self.step * index # next potential result calculated (might never be yielded)
            
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))
ap = ArithmeticProgression(1, .5, 3)
print(list(ap))
ap = ArithmeticProgression(0, 1/3, 1)
print(list(ap))

# with itertools
gen = itertools.count(1, 5)
print(next(gen))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[0.0, 0.3333333333333333, 0.6666666666666666]
1


In [5]:
# generator functions
import itertools

def vowel(c):
    return c.lower() in 'aeiou'

# filtering generator functions
print(list(filter(vowel, 'Aardvark'))) # applies vowel to each item, yielding the item if vowel is true
print(list(itertools.filterfalse(vowel, 'Aardvark'))) # same as filter negated
print(list(itertools.dropwhile(vowel, 'Aardvark'))) # yields items as long as vowel is true, then yields every item
print(list(itertools.compress('Aardvark', (1,0,1,1,0,1)))) # two iterables in parallel: yields items as long as 2nd iterable is true
print(list(itertools.islice('Aardvark', 4))) # yields items from slice
print(list(itertools.islice('Aardvark', 4, 7)))
print(list(itertools.islice('Aardvark', 1, 7, 2))) # start, stop, step

# mapping generator functions
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
print(list(itertools.accumulate(sample))) # running sum

['A', 'a', 'a']
['r', 'd', 'v', 'r', 'k']
['r', 'd', 'v', 'a', 'r', 'k']
['A', 'r', 'd', 'a']
['A', 'a', 'r', 'd']
['v', 'a', 'r']
['a', 'd', 'a']
[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]
